In [40]:
pip install google-genai -q

In [41]:
pip install google-adk -q

In [42]:
print("Olá! Para que o 'terapIA' possa funcionar corretamente e gerar seus relatórios semanais,")
print("precisamos de sua permissão para acessar e criar uma pasta segura no seu Google Drive.")
print("Nessa pasta, será salvo o histórico resumido das nossas conversas.")
print("Seus dados serão armazenados de forma privada e utilizados apenas para o funcionamento do 'terapIA',")
print("permitindo que você acompanhe seus relatos e insights ao longo do tempo.")
print("Por favor, autorize a conexão com o Google Drive quando solicitado.")

Olá! Para que o 'terapIA' possa funcionar corretamente e gerar seus relatórios semanais,
precisamos de sua permissão para acessar e criar uma pasta segura no seu Google Drive.
Nessa pasta, será salvo o histórico resumido das nossas conversas.
Seus dados serão armazenados de forma privada e utilizados apenas para o funcionamento do 'terapIA',
permitindo que você acompanhe seus relatos e insights ao longo do tempo.
Por favor, autorize a conexão com o Google Drive quando solicitado.


In [43]:
### Montar Google Drive

from google.colab import drive
import os
import csv
from datetime import datetime

try:
  drive.mount("/content/drive")
  print("\nGoogle Drive montado com sucesso em '/content/drive'.")
except Exception as e:
  print(f"\nErro ao montar Google Drive: {e}")
  print("Não será possível salvar/carregar dados do Drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Google Drive montado com sucesso em '/content/drive'.


In [44]:
DRIVE_FILE_PATH = "/content/drive/My Drive/terapIA_data/relatos.csv"

DRIVE_FOLDER_PATH = os.path.dirname(DRIVE_FILE_PATH)

print(f"O arquivo de relatos será salvo/lido em: {DRIVE_FILE_PATH}")

O arquivo de relatos será salvo/lido em: /content/drive/My Drive/terapIA_data/relatos.csv


In [45]:
### Importando API Key

import os
import csv
from datetime import datetime, timedelta
import google.generativeai as genai
from google import genai
from google.genai import types
import json
import textwrap
from google.colab import userdata
import ipywidgets
from IPython.display import display

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

### Importando Gemini 2.5 Flash (Preview)

try:
  client = genai.Client()
  modelo = "gemini-2.5-flash-preview-04-17"
except Exception as e:
    print(f"\nErro ao inicializar o cliente Gemini: {e}")
    print("Verifique sua conexão e validade da API Key. Funções do Gemini podem não funcionar.")

### Montando as instruções do ChatBot (Persona e parâmetros de interação)

chat_config = types.GenerateContentConfig(
    system_instruction = """Você é o terapIA, um assistente pessoal e um diário digital para ajudar o usuário a registrar e acompanhar pensamentos, sentimentos e o dia a dia. Seu propósito é ser um espaço seguro, acolhedor e sem julgamentos.

Sua tarefa principal é ajudar o usuário a articular e identificar os seguintes pontos sobre o dia dele:
1.  Uma classificação geral do dia (bom, neutro, ruim).
2.  Os sentimentos experimentados (listando o máximo possível, sejam bons, neutros ou ruins).
3.  Os fatores ou eventos que contribuíram positivamente para o dia ou humor.
4.  Os fatores ou eventos que contribuíram negativamente para o dia ou humor.

Quando o usuário relatar sobre o dia, ouça com empatia e valide os sentimentos. Em seguida, direcione a conversa, de forma suave e com perguntas abertas, para garantir que todos os 4 pontos mencionados acima sejam abordados e claros.

Se o usuário focar apenas em aspectos positivos, pergunte gentilmente sobre desafios, sentimentos difíceis ou algo que não tenha ido tão bem.
Se o usuário focar apenas em aspectos negativos, pergunte gentilmente sobre momentos bons, sentimentos positivos ou algo que tenha ajudado, mesmo que um pouco.
Pergunte explicitamente, se necessário, sobre a classificação geral do dia caso o usuário não mencione.
Quando você conseguir coletar todas as quatro informações, confirme com o usuário se ele tem mais a acrescentar, e sugira que ele responda com "Tchau" quando não houver mais nada a acrescentar, para que os dados possam ser salvos.

Mantenha um tom sempre acolhedor, empático, paciente e encorajador. Use linguagem suave de apoio.
Valide os sentimentos deles usando frases como 'Entendo', 'Isso faz sentido', 'Obrigado por compartilhar isso'.
Não dê conselhos diretos ou soluções; seu foco é ajudar o usuário a explorar e *identificar* os elementos do seu dia para registro.
Evite repetir longos trechos do que o usuário disse, mas mostre que você compreendeu, direcionando a conversa para coletar as informações faltantes (pontos 1 a 4).
"""
)

### Definição Introdução ao ChatBot

def introducao():
  print("Olá, seja bem vindo ao terapIA!\n")

  print("=" * 40)
  print("""AVISO IMPORTANTE
O terapIA é uma ferramenta de apoio para ajudar você a registrar seus dias e pensamentos.
É essencial saber que o terapIA não substitui um psicólogo ou terapeuta profissional.
Ele serve como um auxiliar para o seu processo de autoconhecimento e pode complementar suas sessões de terapia,
mas o acompanhamento profissional é indispensável para sua saúde mental.
Em casos de necessidade, procure sempre a ajuda de um profissional qualificado.""")
  print("=" * 40 + "\n")

### Definição Menu

def menu():
  global opcao_selecionada
  print("=" * 40)
  print("Menu Principal")
  print("O que deseja fazer hoje?")
  print("1 - Informar como foi o meu dia")
  print("2 - Hoje é dia de terapia, quero meu relatório detalhado (últimos 7 dias)")
  print("3 - Hoje é dia de terapia, quero meu resumo semanal e insights (com IA)")
  print("4 - É meu primeiro acesso. O que é o terapIA?")
  print("5 - Sobre o projeto")
  print("6 - Sair")
  print("=" * 40 + "\n")

  while True:
    try:
      opcao_selecionada = int(input("Opção: "))
      if 1 <= opcao_selecionada <= 6:
        break
      else:
        print("Opção inválida. Por favor, escolha um número entre 1 e 6.")
    except ValueError:
      print("Entrada inválida. Por favor, digite um número.")

### Definição Informar sobre o dia

def informar_sobre_o_dia():
  print("\n")
  print("=" * 40)
  print("""Olá! 😊
Que bom te ver por aqui hoje.\n
Estou pronto para te ouvir. Me conta como foi o seu dia?
Tente descrever como você se sentiu e o que você acha que pode ter contribuído para isso -
tanto as coisas boas que te ajudaram, quanto os desafios que surgiram.
Escreva tudo o que sentir vontade em uma única mensagem.
Se lembrar de coisas novas, responda ao chat. \n
Quando terminar de contar sobre o seu dia, é só dizer "Tchau" para eu saber que você acabou, e salvar o registro, ok?\n""")

  # Criando o ChatBot

  try:
    chat_diario = client.chats.create(model = modelo, config = chat_config)
  except Exception as e:
    print(f"\nErro ao iniciar o chat para o dia: {e}")
    print("Não será possível registrar o dia agora. Verifique a conexão com o Gemini.")
    return

  prompt = input("Converse com o terapIA: ")
  print("\n")
  while prompt.lower() != "tchau":
    try:
      resposta = chat_diario.send_message(prompt)
      print("terapIA: ", resposta.text)
      print("\n")
    except Exception as e:
      print(f"Erro ao enviar mensagem para o Gemini: {e}")
      print("Por favor, tente novamente ou digite 'Tchau' para finalizar.")

    prompt = input("Converse com o terapIA: ")
    print("\n")

  print("""Obrigado por compartilhar um pouco do seu dia comigo.
Agora, vou gerar um resumo para guardar suas anotações.\n""")
  print("=" * 40 + "\n")

  # Geração de resumo

  try:
    prompt_resumo = """Baseado na nossa conversa sobre o dia de hoje, forneça um resumo seguindo estritamente o formato abaixo. Não inclua nenhum texto adicional além desta estrutura:

Classificação Geral do Dia: [Classifique o dia como Bom, Neutro ou Ruim baseado na conversa]
Principais Sentimentos Identificados: [Liste os principais sentimentos identificados de forma concisa]
Fatores ou Eventos Positivos: [Liste os fatores ou eventos positivos mencionados]
Fatores ou Eventos Negativos: [Liste os fatores ou eventos negativos ou desafios mencionados]
"""
    resposta_resumo = chat_diario.send_message(prompt_resumo)
    resumo_para_salvar = resposta_resumo.text

    print("--- Resumo do Dia (Gerado pelo terapIA para salvar) ---")
    print(resumo_para_salvar)
    print("----------------------------------------------------")

    salvar_relato_csv(resumo_para_salvar)
  except Exception as e:
    print(f"\nErro ao gerar ou gravar o resumo: {e}")
    print("O relato não foi salvo automaticamente no arquivo CSV.")

    print("""\nEspero que você tenha um bom restante de dia. Até a próxima!\n""")
    print("=" * 40 + "\n")

### Definição Salvar relato

def salvar_relato_csv(resumo_para_salvar):

  data_atual_str = datetime.now().strftime("%Y-%m-%d")

  if not os.path.exists("/content/drive/My Drive"):
    print("\nNão foi possível salvar o relato: Google Drive não está montado.")
    print("Por favor, execute a célula de montagem do Drive e autorize o acesso.")
    return

  # Criação da pasta no Drive

  drive_folder = DRIVE_FOLDER_PATH
  if not os.path.exists(drive_folder):
    print(f"\nPasta '{drive_folder}'não encontrada no Drive. Criando...")
    try:
      os.makedirs(drive_folder, exist_ok=True)
      print(f"Pasta '{drive_folder}' verificada/criada com sucesso!")
    except Exception as e:
      print(f"\nErro ao verificar/criar a pasta '{drive_folder}' no Drive: {e}")
      print("Não será possível salvar o relato.")
      return

  # Escrevendo no arquivo CVS

  try:

    nova_linha = [data_atual_str, resumo_para_salvar]

    file_exists = os.path.exists(DRIVE_FILE_PATH)
    file_is_empty = not file_exists or os.path.getsize(DRIVE_FILE_PATH) == 0

    with open(DRIVE_FILE_PATH, mode="a", newline="", encoding="utf-8") as file:
      writer = csv.writer(file)
      if file_is_empty:
        writer.writerow(["Data", "Resumo Diario"])
        if not file_exists:
          print(f"Arquivo '{DRIVE_FILE_PATH}' criado.")
        else:
          print(f"Arquivo '{DRIVE_FILE_PATH}' estava vazio, adicionando cabeçalho.")

      writer.writerow(nova_linha)

    print (f"\nRelato do dia salvo com sucesso em '{DRIVE_FILE_PATH}'!\n")

  except Exception as e:
    print(f"\nErro ao salvar relato no arquivo CSV: {e}")
    print("Verifique as permissões de acesso ao arquivo no seu Google Drive.")

### Mensagem primeiro acesso

def mensagem_primeiro_acesso():
  print("\n")
  print("=" * 40)
  print("""Olá! Que bom te ver por aqui!

Eu sou o terapIA, seu assistente pessoal para te ajudar a registrar e acompanhar seus pensamentos,
sentimentos e o que acontece no seu dia a dia.

Pense em mim como um diário digital e um amigo que te ajuda a organizar suas ideias.

Como eu posso te ajudar?

1.  Registre seu dia: Você pode me contar sobre como se sente, o que aconteceu, seus desafios ou suas conquistas.
    É só escrever, como se estivesse conversando com um amigo.
2.  Organize seus relatos: Eu vou ajudar a organizar e guardar um resumo das suas anotações para você.
3.  Receba um resumo semanal: No dia da sua terapia, você pode me pedir um resumo de tudo que você registrou, de duas formas:
    Temos um relatório detalhado dos últimos 7 dias e um resumo semanal com insights.
    Isso pode te ajudar a perceber padrões e entender melhor o que influencia seu humor e bem-estar,
    além de você poder compartilhar com seu(sua) terapeuta ou psicólogo(a).

Para começar:

É simples! É só começar a me escrever sobre o seu dia de hoje.
Pode ser qualquer coisa que você queira tirar da cabeça ou registrar.

Estou aqui para te ouvir!""")
  print("=" * 40 + "\n")

### Definição Sobre o terapIA

def sobre_o_projeto():
  print("\n")
  print("=" * 40)
  print("""Sobre o Projeto: terapIA

Este é um projeto de chatbot pessoal criado no Google Colab usando Python
e a tecnologia Google Gemini.

Objetivo Principal:
O "terapIA" foi desenvolvido para ser um espaço seguro e fácil para o usuário
registrar seus relatos diários, pensamentos e sentimentos. A ideia é ajudar
na auto-reflexão e no acompanhamento do humor ao longo do tempo.

Funcionalidades Principais:
1.  Registro Diário: Permite ao usuário escrever livremente sobre o seu dia.
2.  Resumo Inteligente: Utiliza a inteligência artificial (Google Gemini)
    para criar resumos concisos dos relatos diários.
3.  Armazenamento Seguro: Salva os resumos e informações importantes em um
    banco de dados na nuvem.
4.  Resumo Semanal: Possibilita resgatar um resumo consolidado dos relatos
    da semana.
5.  Insights de Humor: Ajuda a identificar e registrar fatores que podem
    ter influenciado o humor do usuário durante a semana.

Tecnologias Utilizadas:
- Linguagem de Programação: Python
- Ambiente de Desenvolvimento: Google Colab
- Modelo de Linguagem/IA: Google Gemini
- Banco de Dados: Arquivo CSV no Google Drive""")
  print("=" * 40 + "\n")

### Definição Gerar o relatório

def extrair_detalhes_do_resumo(resumo_texto):
  # global modelo
  if not modelo:
      print("Modelo Gemini para extração não configurado. Não é possível extrair detalhes.")
      return None
  prompt = f"""Analise o seguinte relato diário e extraia as informações solicitadas.
  Forneça a saída estritamente no formato JSON.
  Se uma informação não estiver presente ou não for clara, use "Não informado"
  para strings ou [] para listas vazias.

Relato Diário:
{resumo_texto}

Informações Requisitadas (apenas JSON):
{{
  "humor_geral": "Bom, Neutro, ou Ruim",
  "sentimentos": ["sentimento1", "sentimento2", ...],
  "pontos_positivos": ["ponto positivo1", "ponto positivo2", ...],
  "pontos_negativos": ["ponto negativo1", "ponto negativo2", ...]
}}
"""

  try:
    chat_resumo = client.chats.create(model = modelo)
    response = chat_resumo.send_message(prompt)
    response_text = response.text.strip()
    if response_text.startswith('```json'):
      response_text = response_text[len('```json'):]
      if response_text.endswith('```'):
        response_text = response_text[:-len('```')]
    response_text = response_text.strip()

    detalhes_extraidos = json.loads(response_text)
    return detalhes_extraidos

  except json.JSONDecodeError as e:
    print(f"Erro ao analisar a resposta JSON do modelo: {e}")
    print(f"Resposta recebida (tentando analisar): {response_text}")
    return None
  except Exception as e:
    print(f"Erro ao chamar o modelo Gemini para extração: {e}")
    return None


def gerar_relatorio_detalhado_semanal_com_extracao():
    print("\n")
    print("=" * 40 + "")
    print("--- Relatório Detalhado Semanal (com Extração por IA) ---")

    # Verificar se o Google Drive está montado
    if not os.path.exists('/content/drive/My Drive'):
      print("\nNão foi possível gerar o relatório: Google Drive não está montado.")
      print("Por favor, execute a célula de montagem do drive e autorize o acesso.")
      return

    # Verificar se o arquivo de relatos existe
    if not os.path.exists(DRIVE_FILE_PATH):
      print(f"\nNenhum relato encontrado: O arquivo '{DRIVE_FILE_PATH}' não existe ainda.")
      print("Salve alguns relatos primeiro para gerar um relatório.")
      return

    # Verificar se o modelo Gemini para extração está configurado
    global modelo
    if not modelo:
      print("\nO modelo Gemini para extração não foi configurado corretamente. Não é possível gerar o relatório que requer extração.")
      print("Por favor, verifique sua GOOGLE_API_KEY e a configuração do modelo.")
      return

    try:
      relatos_completos = []
      # Verificar se o arquivo está vazio antes de tentar ler
      if os.path.getsize(DRIVE_FILE_PATH) == 0:
        print("O arquivo de relatos existe, mas está vazio.")
        return

      # Ler os dados do arquivo CSV (apenas Data e Resumo)
      with open(DRIVE_FILE_PATH, mode="r", newline="", encoding="utf-8") as file:
        reader = csv.reader(file)
        # Tentar pular o cabeçalho, se houver
        try:
          header = next(reader)
          if len(header) < 2 or header[0].strip() != 'Data' or header[1].strip() != 'Resumo Diario':
            print("Aviso: A estrutura do arquivo CSV pode não ser a esperada (esperado: 'Data','Resumo Diario').")

        except StopIteration:
          print("O arquivo CSV está vazio (não tem nem cabeçalho).")
          return

        for row in reader:
          if row and len(row) >= 2:
            relatos_completos.append(row)
          elif row:
            if ''.join(row).strip():
              print(f"Aviso: Linha com formato inesperado encontrada (menos de 2 colunas): {row}. Ignorando.")


      # Filtrar relatos dos últimos 7 dias
      hoje = datetime.now().date()
      sete_dias_atras = hoje - timedelta(days=6)

      relatos_ultimos_7_dias = []
      for relato in relatos_completos:
        try:
          data_relato_str = relato[0]
          data_relato_obj = datetime.strptime(data_relato_str, '%Y-%m-%d').date()

          if data_relato_obj >= sete_dias_atras and data_relato_obj <= hoje:
            relatos_ultimos_7_dias.append(relato)

        except ValueError:
          print(f"Aviso: Não foi possível processar a data '{relato[0]}' (formato inesperado) na linha: {relato}. Ignorando esta linha.")
        except IndexError:
          pass

      # Verificar se há relatos nos últimos 7 dias
      if not relatos_ultimos_7_dias:
        print("Nenhum relato encontrado nos últimos 7 dias para gerar o relatório.")
        return

      # Ordenar os relatos por data (do mais antigo para o mais novo)
      relatos_ultimos_7_dias.sort(key=lambda r: datetime.strptime(r[0], '%Y-%m-%d'))


      # Exibir o relatório formatado, extraindo detalhes com IA
      print(f"\nExibindo relatos dos últimos {len(relatos_ultimos_7_dias)} dias (última semana):\n")

      for i, relato in enumerate(relatos_ultimos_7_dias):
        data = relato[0]
        resumo_texto = relato[1] # O texto completo do resumo diário

        print(f"--- Dia {i+1} ({data}) ---")

        detalhes_extraidos = extrair_detalhes_do_resumo(resumo_texto)

        if detalhes_extraidos:
          humor = detalhes_extraidos.get("humor_geral", "Não informado")
          sentimentos = ", ".join(detalhes_extraidos.get("sentimentos", [])) if detalhes_extraidos.get("sentimentos") else "Não informados"
          pontos_positivos = ", ".join(detalhes_extraidos.get("pontos_positivos", [])) if detalhes_extraidos.get("pontos_positivos") else "Não informados"
          pontos_negativos = ", ".join(detalhes_extraidos.get("pontos_negativos", [])) if detalhes_extraidos.get("pontos_negativos") else "Não informados"

          print(f"Como foi o dia: {humor}")
          print(f"Sentimentos: {sentimentos}")
          print(f"Pontos positivos: {pontos_positivos}")
          print(f"Pontos negativos: {pontos_negativos}")
        else:
          print("Não foi possível extrair detalhes deste relato.")

        print("-" * 20) # Separador entre os dias

      print("--- Fim do Relatório Detalhado Semanal ---")
      print("=" * 40)
      print("\n")

    except Exception as e:
      print(f"\nErro inesperado ao gerar o relatório detalhado: {e}")
      print("Verifique o arquivo CSV, a definição de DRIVE_FILE_PATH e a configuração da API Gemini.")

### Relatório de Resumo Semanal com IA

def gerar_relatorio_resumo_semanal_com_ia():
    print("\n")
    print("=" * 40 + "")
    print("--- Relatório de Resumo Semanal e Insights (com IA) ---")

    # Verificar se o Google Drive está montado
    if not os.path.exists('/content/drive/My Drive'):
        print("\nNão foi possível gerar o relatório: Google Drive não está montado.")
        print("Por favor, execute a célula de montagem do drive e autorize o acesso.")
        return

    # Verificar se o arquivo de relatos existe
    if not os.path.exists(DRIVE_FILE_PATH):
        print(f"\nNenhum relato encontrado: O arquivo '{DRIVE_FILE_PATH}' não existe ainda.")
        print("Salve alguns relatos primeiro para gerar um relatório.")
        return

    global modelo
    if not modelo:
         print("\nO modelo Gemini para relatórios não foi configurado corretamente. Não é possível gerar o relatório.")
         print("Por favor, verifique sua GOOGLE_API_KEY e a configuração do modelo.")
         return

    try:
        relatos_completos = []
        if os.path.getsize(DRIVE_FILE_PATH) == 0:
            print("O arquivo de relatos existe, mas está vazio.")
            return

        with open(DRIVE_FILE_PATH, mode="r", newline="", encoding="utf-8") as file:
            reader = csv.reader(file)
            try:
                header = next(reader)
                # Validação do cabeçalho para a estrutura Data, Resumo Diario
                if len(header) < 2 or header[0].strip() != 'Data' or header[1].strip() != 'Resumo Diario':
                   print("Aviso: A estrutura do arquivo CSV pode não ser a esperada (esperado: 'Data','Resumo Diario'). O relatório pode sair incorreto.")
            except StopIteration:
                print("O arquivo CSV está vazio (não tem nem cabeçalho).")
                return

            for row in reader:
                if row and len(row) >= 2:
                    relatos_completos.append(row)
                elif row:
                     if ''.join(row).strip():
                        print(f"Aviso: Linha com formato inesperado encontrada (menos de 2 colunas): {row}. Ignorando.")


        hoje = datetime.now().date()
        sete_dias_atras = hoje - timedelta(days=6) # Inclui hoje e os 6 dias anteriores

        relatos_ultimos_7_dias = []
        for relato in relatos_completos:
            try:
                data_relato_str = relato[0]
                data_relato_obj = datetime.strptime(data_relato_str, '%Y-%m-%d').date()

                if data_relato_obj >= sete_dias_atras and data_relato_obj <= hoje:
                     relatos_ultimos_7_dias.append(relato)

            except ValueError:
                print(f"Aviso: Não foi possível processar a data '{relato[0]}' (formato inesperado) na linha: {relato}. Ignorando esta linha.")
            except IndexError:
                 pass


        if not relatos_ultimos_7_dias:
            print("Nenhum relato encontrado nos últimos 7 dias para gerar o resumo semanal.")
            return

        # Ordenar os relatos por data (do mais antigo para o mais novo)
        relatos_ultimos_7_dias.sort(key=lambda r: datetime.strptime(r[0], '%Y-%m-%d'))

        # --- Preparar o texto para enviar para a IA ---
        texto_combinado_relatos = ""
        for relato in relatos_ultimos_7_dias:
            data = relato[0]
            resumo = relato[1]
            texto_combinado_relatos += f"--- Relato de {data} ---\n{resumo}\n\n"

        # --- Prompt para a IA gerar o Resumo Semanal e Insights ---
        prompt_resumo_semanal = f"""Com base nos seguintes relatos diários da última semana, forneça um resumo consolidado e insights:

Relatos Diários (últimos {len(relatos_ultimos_7_dias)} dias):
{texto_combinado_relatos}

Por favor, gere um relatório que inclua:

1. Um resumo conciso da evolução da semana.
2. Um resumo conciso da evolução do humor e principais eventos ao longo da semana.
3. Uma análise dos fatores positivos mais citados, mencionando os temas ou eventos que mais se repetiram ou tiveram maior impacto.
4. Uma análise dos fatores negativos mais citados, mencionando os temas ou eventos que mais se repetiram ou tiveram maior impacto.
5. 3 a 5 insights ou observações gerais sobre a semana, aprendizados ou padrões que possam ter surgido nos relatos.

Formate a resposta de forma clara, usando títulos para cada seção e tópicos.
"""
        print("terapIA: Analisando a semana para gerar o resumo e insights...")

        # --- Chamar o modelo Gemini para gerar o relatório ---
        try:
            chat_resumo_semanal = client.chats.create(model = modelo)
            response = chat_resumo_semanal.send_message(prompt_resumo_semanal)
            response_text = response.text.strip()

            print("\n--- Resumo da Sua Semana no terapIA ---\n")
            print(response_text)
            print("\n--- Fim do Relatório Semanal ---")
            print("=" * 40 + "\n")


        except Exception as e:
            print(f"\nErro ao gerar o resumo semanal com a IA: {e}")
            print("Não foi possível obter o relatório da semana.")

    except Exception as e:
        print(f"\nErro inesperado ao ler dados para o resumo semanal: {e}")
        print("Verifique o arquivo CSV e a definição de DRIVE_FILE_PATH.")

### Introdução

introducao()

### Menu

opcao_selecionada = 0
opcao_sair = 6
while opcao_selecionada != opcao_sair:
  menu()
  if(opcao_selecionada == 1):
   informar_sobre_o_dia()
  elif(opcao_selecionada == 2):
   gerar_relatorio_detalhado_semanal_com_extracao()
  elif(opcao_selecionada == 3):
    gerar_relatorio_resumo_semanal_com_ia()
  elif(opcao_selecionada == 4):
   mensagem_primeiro_acesso()
  elif(opcao_selecionada == 5):
   sobre_o_projeto()
  elif(opcao_selecionada == opcao_sair):
    print("\nSaindo do terapIA. Espero te ver novamente em breve!")
  else:
    print("\nOpção inválida. Por favor, escolha uma opção do menu.")

print("\n\nPrograma terapIA finalizado.")


Olá, seja bem vindo ao terapIA!

AVISO IMPORTANTE
O terapIA é uma ferramenta de apoio para ajudar você a registrar seus dias e pensamentos.
É essencial saber que o terapIA não substitui um psicólogo ou terapeuta profissional.
Ele serve como um auxiliar para o seu processo de autoconhecimento e pode complementar suas sessões de terapia,
mas o acompanhamento profissional é indispensável para sua saúde mental.
Em casos de necessidade, procure sempre a ajuda de um profissional qualificado.

Menu Principal
O que deseja fazer hoje?
1 - Informar como foi o meu dia
2 - Hoje é dia de terapia, quero meu relatório detalhado (últimos 7 dias)
3 - Hoje é dia de terapia, quero meu resumo semanal e insights (com IA)
4 - É meu primeiro acesso. O que é o terapIA?
5 - Sobre o projeto
6 - Sair



KeyboardInterrupt: Interrupted by user